# Summing warehouse businesses by zip code with ZBP data

DT

4/20/22

This requires the zip code data to be unzipped.
ZBP data is in the same folders as the CBPs.

In [6]:
import pandas as pd
import os

In [7]:
os.listdir('../raw_data/cbp19/')

['cbp19cd.xlsx',
 'cbp19co.zip',
 'cbp19csa.zip',
 'cbp19msa.zip',
 'cbp19pr_ia_co.zip',
 'cbp19pr_ia_st.zip',
 'cbp19st.zip',
 'cbp19us.zip',
 'zbp19detail.txt',
 'zbp19detail.zip',
 'zbp19totals.zip']

In [8]:
%%time
zbp19 = pd.read_csv('../raw_data/cbp19/zbp19detail.txt', encoding = 'ISO-8859-1')
zbp19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2881311 entries, 0 to 2881310
Data columns (total 16 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   zip       int64 
 1   name      object
 2   naics     object
 3   est       int64 
 4   n<5       object
 5   n5_9      object
 6   n10_19    object
 7   n20_49    object
 8   n50_99    object
 9   n100_249  object
 10  n250_499  object
 11  n500_999  object
 12  n1000     object
 13  city      object
 14  stabbr    object
 15  cty_name  object
dtypes: int64(2), object(14)
memory usage: 351.7+ MB
Wall time: 4.85 s


In [9]:
zbp19['zip'].value_counts()

33166    830
10001    816
92705    760
11101    746
80112    729
        ... 
72812      1
28272      1
72776      1
72769      1
501        1
Name: zip, Length: 35129, dtype: int64

In [10]:
zbp19['naics'].value_counts()[:20]

------    35129
81----    22504
23----    22427
44----    21916
238///    19475
72----    19076
813///    18897
722///    18117
62----    17955
541///    17748
54----    17748
72251/    16735
7225//    16735
56----    16539
813110    16416
8131//    16416
81311/    16416
48----    16118
561///    15812
31----    15775
Name: naics, dtype: int64

There are 35129 total businesses.
The big ones:
- 81----    is Other Services.
- 23----    is construction. (This could be useful when thinking about warehouses?? How do construction business work in terms of establishments vs sites?)
- 44----	Retail Trade
- 238///	Specialty Trade Contractors

More in the NAICS notebook. The ones we're focused on are the warehouses.

Warehouse NAICS:
Our codes are `493///` for Warehousing and Storage.
`4931//` is the only subset of the above. (like taxonomy -- _sapiens_ is the only living species in _homo_)
- `49311/`, `493110` General
- `49312/`, `493120` Refrigerated
- `49313/`, `493130` Farm Products
- `49319/`, `493190` Other


It will be helpful to first just extract only the rows with zips in cali.

4/21/22

made a couple dictionaries to translate tracts to zips and vice versa.

In [12]:
import json

dicts_filepath = '../CA_zip_tract_translation/'

tract_to_zip_dict_filepath = dicts_filepath + 'tract_to_zip.json'
zip_to_tract_dict_filepath = dicts_filepath + 'zip_to_tract.json'

with open (tract_to_zip_dict_filepath, 'r') as my_ttz_file:
    tract_to_zip = json.load(my_ttz_file)

with open (zip_to_tract_dict_filepath, 'r') as my_ztt_file:
    zip_to_tract = json.load(my_ztt_file)

In [13]:
zip_to_tract['90210']

[6037700801, 6037700700, 6037141700, 6037261101, 6037700600, 6037261102]

In [14]:
tract_to_zip['6037700801']

90210

I'd like a DataFrame with observations like the following...


| index | census tract |  zip code | original CAES number | total business type 1 | total business type 2 | ... |
| --- | --- | --- | --- | --- | --- | --- |

making sure not to use census tract or zip as indices --- there will be multiple ones from each CAES set.

In [15]:
all_cali_zips = set(zip_to_tract.keys())
all_cali_zips

{'94544',
 '96067',
 '92802',
 '93105',
 '95623',
 '94519',
 '94608',
 '94954',
 '93066',
 '91306',
 '95585',
 '95698',
 '92707',
 '93311',
 '93647',
 '92866',
 '92058',
 '91906',
 '95948',
 '95134',
 '92129',
 '95126',
 '91786',
 '95236',
 '90056',
 '90290',
 '94505',
 '95916',
 '95821',
 '95425',
 '92252',
 '93955',
 '93247',
 '94598',
 '94956',
 '94587',
 '92119',
 '92009',
 '95257',
 '90041',
 '95638',
 '92683',
 '92596',
 '93546',
 '94579',
 '91915',
 '95691',
 '93445',
 '95987',
 '92507',
 '95667',
 '90723',
 '95210',
 '93552',
 '93458',
 '91755',
 '95969',
 '90502',
 '94070',
 '90810',
 '96064',
 '90058',
 '91402',
 '90062',
 '90038',
 '93430',
 '96035',
 '91739',
 '93257',
 '92010',
 '95212',
 '91803',
 '94014',
 '92113',
 '95746',
 '96003',
 '92629',
 '94574',
 '92027',
 '92081',
 '95682',
 '95713',
 '91752',
 '95838',
 '94518',
 '95490',
 '91732',
 '92503',
 '93640',
 '92625',
 '90077',
 '95837',
 '95120',
 '92127',
 '95684',
 '90040',
 '93706',
 '90270',
 '95823',
 '95503',


In [16]:
#there are almost 300000 observations from california in zbp19.

#make a table of bools to mask the non-calis
zbp19_cali_truths = zbp19['zip'].apply(lambda zip_code: str(zip_code) in all_cali_zips)
zbp19_cali_truths.sum()

299337

In [20]:
#mask the non-calis with that truth table.
zbp19_cali = zbp19[zbp19_cali_truths]
zbp19_cali

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2443695,90001,"LOS ANGELES, CA",------,613,316,111,96,73,4,12,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443696,90001,"LOS ANGELES, CA",23----,28,13,8,6,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443697,90001,"LOS ANGELES, CA",236///,6,4,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443698,90001,"LOS ANGELES, CA",2361//,5,4,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443699,90001,"LOS ANGELES, CA",23611/,5,4,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749280,96161,"TRUCKEE, CA",81331/,8,5,N,N,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749281,96161,"TRUCKEE, CA",813312,7,4,N,N,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749282,96161,"TRUCKEE, CA",8139//,11,4,N,3,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749283,96161,"TRUCKEE, CA",81399/,9,4,N,3,N,N,N,N,N,N,TRUCKEE,CA,NEVADA


In [21]:
zbp19_cali.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299337 entries, 2443695 to 2749284
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   zip       299337 non-null  int64 
 1   name      299337 non-null  object
 2   naics     299337 non-null  object
 3   est       299337 non-null  int64 
 4   n<5       299337 non-null  object
 5   n5_9      299337 non-null  object
 6   n10_19    299337 non-null  object
 7   n20_49    299337 non-null  object
 8   n50_99    299337 non-null  object
 9   n100_249  299337 non-null  object
 10  n250_499  299337 non-null  object
 11  n500_999  299337 non-null  object
 12  n1000     299337 non-null  object
 13  city      299337 non-null  object
 14  stabbr    299337 non-null  object
 15  cty_name  299337 non-null  object
dtypes: int64(2), object(14)
memory usage: 38.8+ MB


In [22]:
# look at construction. There are some with '-' and some with '/' as their placeholders.
# I'm not sure there's a difference between the two symbols.
zbp19_cali[zbp19_cali['naics'] == '23----']

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2443696,90001,"LOS ANGELES, CA",23----,28,13,8,6,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443959,90002,"LOS ANGELES, CA",23----,21,17,N,3,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2444047,90003,"LOS ANGELES, CA",23----,28,22,4,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2444196,90004,"LOS ANGELES, CA",23----,50,34,12,3,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2444565,90005,"LOS ANGELES, CA",23----,34,27,5,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2748356,96142,"TAHOMA, CA",23----,14,11,N,N,N,N,N,N,N,N,TAHOMA,CA,EL DORADO
2748368,96143,"KINGS BEACH, CA",23----,29,23,4,N,N,N,N,N,N,N,KINGS BEACH,CA,PLACER
2748421,96145,"TAHOE CITY, CA",23----,60,40,14,5,N,N,N,N,N,N,TAHOE CITY,CA,PLACER
2748648,96150,"SOUTH LAKE TAHOE, CA",23----,154,124,20,8,N,N,N,N,N,N,SOUTH LAKE TAHOE,CA,EL DORADO


In [23]:
#221 "summary" rows with ALL warehouses in each zip. 
zbp19_cali[zbp19_cali['naics'] == '493///']

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2443850,90001,"LOS ANGELES, CA",493///,3,N,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2449655,90021,"LOS ANGELES, CA",493///,9,4,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2450345,90023,"LOS ANGELES, CA",493///,14,3,N,N,4,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2455713,90040,"LOS ANGELES, CA",493///,16,5,3,3,N,3,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2456997,90045,"LOS ANGELES, CA",493///,6,N,3,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739721,95834,"SACRAMENTO, CA",493///,4,N,N,N,N,N,N,N,N,N,SACRAMENTO,CA,SACRAMENTO
2740236,95838,"SACRAMENTO, CA",493///,3,N,N,N,N,N,N,N,N,N,SACRAMENTO,CA,SACRAMENTO
2742450,95945,"GRASS VALLEY, CA",493///,3,N,N,N,N,N,N,N,N,N,GRASS VALLEY,CA,NEVADA
2744460,95987,"WILLIAMS, CA",493///,7,5,N,N,N,N,N,N,N,N,WILLIAMS,CA,COLUSA


In [24]:
#make a dict for converting warehouse types to codes.
warehouse_dict = {
    'warehouses':'493///',
    'gen/':'49311/',
    'gen':'493110',
    'cold':'49312/',
    'farm/':'49313/',
    'farm':'493130',
    'other/':'49319/',
    'other':'493190'
}

In [25]:
#try it out.
df2 = zbp19_cali[zbp19_cali['naics'] == warehouse_dict['gen'] ]

In [26]:
# the general code has the same statistics as the more specific code.

df1 = zbp19_cali[zbp19_cali['naics'] == warehouse_dict['gen/']]

In [31]:
#just checking if dividing by 'gen' type is the same as dividing by 'gen/' type.
df1.drop(columns = 'naics').values == df2.drop(columns = 'naics').values

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [28]:
sum(_)

array([174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174,
       174, 174])

In [29]:
len(df1)

174

In [30]:
# ^ They are the same. ^

In [36]:
df1 = zbp19_cali[zbp19_cali['naics'] == warehouse_dict['cold']]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 2449659 to 2715469
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   zip       20 non-null     int64 
 1   name      20 non-null     object
 2   naics     20 non-null     object
 3   est       20 non-null     int64 
 4   n<5       20 non-null     object
 5   n5_9      20 non-null     object
 6   n10_19    20 non-null     object
 7   n20_49    20 non-null     object
 8   n50_99    20 non-null     object
 9   n100_249  20 non-null     object
 10  n250_499  20 non-null     object
 11  n500_999  20 non-null     object
 12  n1000     20 non-null     object
 13  city      20 non-null     object
 14  stabbr    20 non-null     object
 15  cty_name  20 non-null     object
dtypes: int64(2), object(14)
memory usage: 2.7+ KB


In [37]:
df1

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2449659,90021,"LOS ANGELES, CA",49312/,4,N,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2450349,90023,"LOS ANGELES, CA",49312/,6,N,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2459476,90058,"LOS ANGELES, CA",49312/,17,5,N,N,7,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2481757,90640,"MONTEBELLO, CA",49312/,3,N,N,N,N,N,N,N,N,N,MONTEBELLO,CA,LOS ANGELES
2487475,90744,"WILMINGTON, CA",49312/,3,N,N,N,N,N,N,N,N,N,WILMINGTON,CA,LOS ANGELES
2519220,91706,"BALDWIN PARK, CA",49312/,3,N,N,N,N,N,N,N,N,N,BALDWIN PARK,CA,LOS ANGELES
2521921,91730,"RANCHO CUCAMONGA, CA",49312/,3,N,N,N,N,N,N,N,N,N,RANCHO CUCAMONGA,CA,SAN BERNARDINO
2527528,91761,"ONTARIO, CA",49312/,6,N,N,3,N,N,N,N,N,N,ONTARIO,CA,SAN BERNARDINO
2567685,92335,"FONTANA, CA",49312/,3,N,N,N,N,N,N,N,N,N,FONTANA,CA,SAN BERNARDINO
2575519,92509,"RIVERSIDE, CA",49312/,5,N,N,N,N,N,N,N,N,N,RIVERSIDE,CA,RIVERSIDE


20 entries in the refrigerated class.

In [35]:
zbp19_cali[zbp19_cali['naics'] == warehouse_dict['farm']]

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2617853,93212,"CORCORAN, CA",493130,3,N,N,N,N,N,N,N,N,N,CORCORAN,CA,KINGS
2639014,93725,"FRESNO, CA",493130,3,N,N,N,N,N,N,N,N,N,FRESNO,CA,FRESNO


Only two in the farm type.

In [38]:
zbp19_cali[zbp19_cali['naics'] == warehouse_dict['other']]

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2459479,90058,"LOS ANGELES, CA",493190,5,N,3,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2482512,90660,"PICO RIVERA, CA",493190,6,N,N,N,N,N,N,N,N,N,PICO RIVERA,CA,LOS ANGELES
2483086,90670,"SANTA FE SPRINGS, CA",493190,4,N,N,N,N,N,N,N,N,N,SANTA FE SPRINGS,CA,LOS ANGELES
2484013,90703,"CERRITOS, CA",493190,3,N,N,N,N,N,N,N,N,N,CERRITOS,CA,LOS ANGELES
2487742,90745,"CARSON, CA",493190,3,N,N,N,N,N,N,N,N,N,CARSON,CA,LOS ANGELES
2491144,90813,"LONG BEACH, CA",493190,3,N,N,N,N,N,N,N,N,N,LONG BEACH,CA,LOS ANGELES
2525722,91748,"ROWLAND HEIGHTS, CA",493190,5,N,N,N,N,N,N,N,N,N,ROWLAND HEIGHTS,CA,LOS ANGELES
2527531,91761,"ONTARIO, CA",493190,6,N,N,N,N,N,N,N,N,N,ONTARIO,CA,SAN BERNARDINO
2529444,91766,"POMONA, CA",493190,3,N,N,N,N,N,N,N,N,N,POMONA,CA,LOS ANGELES
2537631,91950,"NATIONAL CITY, CA",493190,3,N,N,N,N,N,N,N,N,N,NATIONAL CITY,CA,SAN DIEGO


# counting and combining

PLAN:
- get a list of all unique zips
    - that's easy. I already have it from the zip to tract dictionary.
- create a new dataframe
- for each zip,
    - create an observation for each zip / year with est total, est gen, est farm, est cold, est other.
    - boolean mask out all the observations with that zip
    - for each observation, add est to gen/farm/cold/other total
    

In [67]:
columns = ['zip', 'year', 'est total', 'est gen', 'est cold', 'est farm', 'est other']
zip_df = pd.DataFrame(columns = columns)

In [68]:
zip_df['zip'] = zip_to_tract.keys()
zip_df['year'] = 2019
zip_df

,zip,year,est total,est gen,est cold,est farm,est other
0,93706,2019,NaN,NaN,NaN,NaN,NaN
1,91761,2019,NaN,NaN,NaN,NaN,NaN
2,95203,2019,NaN,NaN,NaN,NaN,NaN
3,93725,2019,NaN,NaN,NaN,NaN,NaN
4,90023,2019,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1350,95389,2019,NaN,NaN,NaN,NaN,NaN
1351,94964,2019,NaN,NaN,NaN,NaN,NaN
1352,95720,2019,NaN,NaN,NaN,NaN,NaN
1353,57,2019,NaN,NaN,NaN,NaN,NaN


In [69]:
filter_dict = {
    'total':'493///',
    'gen':'493110',
    'cold':'49312/',
    'farm':'493130',
    'other':'493190' 
}

In [96]:
%%time
#takes a while. there's probably a much more efficient way of doing this, butcha kno

for index in df19.index:
    if index%100 == 0:
        print(f'working on index {index}')
    this_zip = int(zip_df.loc[index, 'zip'])
    this_df = zbp19_cali[zbp19_cali['zip'] == this_zip]
    for key in filter_dict:
        zip_df[zip_df['est ' + key ]] = this_df[this_df['naics'] == filter_dict[key]]['est'].sum() #add up all the ests for this zip/naics pair

working on index 0


KeyboardInterrupt: 

In [66]:
for thing in filter_dict:
    print(thing)

total
gen
cold
farm
other


In [72]:
zip_df

,zip,year,est total,est gen,est cold,est farm,est other,NaN
0,93706,2019,NaN,NaN,NaN,NaN,NaN,0
1,91761,2019,NaN,NaN,NaN,NaN,NaN,0
2,95203,2019,NaN,NaN,NaN,NaN,NaN,0
3,93725,2019,NaN,NaN,NaN,NaN,NaN,0
4,90023,2019,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...
1350,95389,2019,NaN,NaN,NaN,NaN,NaN,0
1351,94964,2019,NaN,NaN,NaN,NaN,NaN,0
1352,95720,2019,NaN,NaN,NaN,NaN,NaN,0
1353,57,2019,NaN,NaN,NaN,NaN,NaN,0


In [73]:
df19.index

RangeIndex(start=0, stop=1355, step=1)

In [76]:
df19.loc[0,'zip']

'93706'

In [94]:
this_zip = zip_df.loc[0,'zip']
zbp19_cali[zbp19_cali['zip'] == 96161]

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2749002,96161,"TRUCKEE, CA",------,901,549,175,100,53,17,4,N,N,N,TRUCKEE,CA,NEVADA
2749003,96161,"TRUCKEE, CA",23----,212,141,36,22,11,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749004,96161,"TRUCKEE, CA",236///,100,71,12,8,8,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749005,96161,"TRUCKEE, CA",2361//,98,70,12,7,8,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749006,96161,"TRUCKEE, CA",23611/,98,70,12,7,8,N,N,N,N,N,TRUCKEE,CA,NEVADA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749280,96161,"TRUCKEE, CA",81331/,8,5,N,N,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749281,96161,"TRUCKEE, CA",813312,7,4,N,N,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749282,96161,"TRUCKEE, CA",8139//,11,4,N,3,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749283,96161,"TRUCKEE, CA",81399/,9,4,N,3,N,N,N,N,N,N,TRUCKEE,CA,NEVADA


In [99]:
zbp19_cali[zbp19_cali['naics'] == filter_dict['gen']]

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2449658,90021,"LOS ANGELES, CA",493110,3,N,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2450348,90023,"LOS ANGELES, CA",493110,7,N,N,N,3,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2455716,90040,"LOS ANGELES, CA",493110,15,5,3,3,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2457000,90045,"LOS ANGELES, CA",493110,6,N,3,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2459475,90058,"LOS ANGELES, CA",493110,15,5,N,N,4,N,4,N,N,N,LOS ANGELES,CA,LOS ANGELES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739724,95834,"SACRAMENTO, CA",493110,3,N,N,N,N,N,N,N,N,N,SACRAMENTO,CA,SACRAMENTO
2740239,95838,"SACRAMENTO, CA",493110,3,N,N,N,N,N,N,N,N,N,SACRAMENTO,CA,SACRAMENTO
2742453,95945,"GRASS VALLEY, CA",493110,3,N,N,N,N,N,N,N,N,N,GRASS VALLEY,CA,NEVADA
2744463,95987,"WILLIAMS, CA",493110,7,5,N,N,N,N,N,N,N,N,WILLIAMS,CA,COLUSA


In [92]:
zbp19_cali

,zip,name,naics,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,city,stabbr,cty_name
2443695,90001,"LOS ANGELES, CA",------,613,316,111,96,73,4,12,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443696,90001,"LOS ANGELES, CA",23----,28,13,8,6,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443697,90001,"LOS ANGELES, CA",236///,6,4,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443698,90001,"LOS ANGELES, CA",2361//,5,4,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
2443699,90001,"LOS ANGELES, CA",23611/,5,4,N,N,N,N,N,N,N,N,LOS ANGELES,CA,LOS ANGELES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749280,96161,"TRUCKEE, CA",81331/,8,5,N,N,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749281,96161,"TRUCKEE, CA",813312,7,4,N,N,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749282,96161,"TRUCKEE, CA",8139//,11,4,N,3,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
2749283,96161,"TRUCKEE, CA",81399/,9,4,N,3,N,N,N,N,N,N,TRUCKEE,CA,NEVADA
